# Agents

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/extras/use_cases/agents/agents.ipynb)

## Use case 

---

LLM-based agents are powerful problem solvers.

The [primary LLM agent components](https://lilianweng.github.io/posts/2023-06-23-agent/) include at least 3 things:

* `Planning`: The ability to break down tasks into smaller sub-goals
* `Memory`: The ability to retain and recall information
* `Tools`: The ability to get information from external sources (e.g., APIs)

![Image description](/img/agents_use_case_1.png)


## Overview 

--- 
 
**Planning**
 
* `Prompt`: Can given the LLM personality, context, or strategies for learninng
* `Agent` Responsible for deciding what step to take next using LLM and the `Prompt`

**Memory**

* This can be short or long-term, allowing the agent to persist information.

**Tools**

* Tools are functions that an agent can call.


## Quickstart 

--- 

In [ ]:
! pip install openai 
! pip install langchain
! pip install google-search-results

import os
os.environ["OPENAI_API_KEY"] = "your-api-key"
# Optional
os.environ["SERPAPI_API_KEY"] = "your-api-key"

`Tool`

LangChain has [many tools](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/load_tools.py) for Agents that we can load easily.

In [ ]:
# Tool
from langchain.agents import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

`Agent`

LangChain has [many agents](https://python.langchain.com/docs/modules/agents/agent_types/).

The [`OPENAI_FUNCTIONS` agent](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent) is a good one to start with.

OpenAI models have been fine-tuned to recognize when function should be called.


In [86]:
# Prompt
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are a search assistant.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

# Agent
search_agent = OpenAIFunctionsAgent(llm=llm, tools=search_tool, prompt=prompt)
agent_executor = AgentExecutor(agent=search_agent, tools=search_tool, verbose=False)

In [87]:
# Run
agent_executor.run("How many people live in canada as of 2023?")

'The current population of Canada is 38,808,843 as of Tuesday, August 1, 2023, based on Worldometer elaboration of the latest United Nations data 1. Canada 2023\xa0... Mar 22, 2023 ... Record-high population growth in the year 2022. Canada\'s population was estimated at 39,566,248 on January 1, 2023, after a record population\xa0... Jun 19, 2023 ... As of June 16, 2023, there are now 40 million Canadians! This is a historic milestone for Canada and certainly cause for celebration. It is also\xa0... Jun 28, 2023 ... Canada\'s population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023. This was the\xa0... Jun 16, 2023 ... Canada\'s population clock (real-time model) ... Consult "Differences between Statistics Canada\'s census counts and ... (April 1, 2023). The current population of Canada in 2023 is 38,781,291, a 0.85% increase from 2022. The population of Canada in 2022 was 38,454,327, a 0.78% increase from 2021. Jun 28, 2023 ...

Great, we have created a simple search agent with a tool!

Looking at the LangSmith [trace](https://smith.langchain.com/public/b7c627dc-cf29-47ef-bc1f-9e4eddeb6c2b/r) we can see exactly what is going on: ChatOpenAI creates a function call to the search tool.

![Image description](/img/agents_use_case_trace_1.png)

## Memory 

--- 

### Short-term memory

Of course, `memory` is needed to enable conversation / persistence of information.

LangChain has many options for [short-term memory](https://python.langchain.com/docs/modules/memory/types/), which are frequently used in [chat](https://python.langchain.com/docs/modules/memory/adding_memory.html). They can be [employed with agents](https://python.langchain.com/docs/modules/memory/agent_with_memory) too.

`ConversationBufferMemory` is a popular choice for short-term memory.

We set `MEMORY_KEY`, which can be referenced by the prompt later.

In [88]:
from langchain.memory import ConversationBufferMemory
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

Now, let's add memory to our agent.

In [93]:
# Prompt w/ placeholder for memory
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
system_message = SystemMessage(content="You are a search assistant.")
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

# Agent
search_agent_memory = OpenAIFunctionsAgent(llm=llm, tools=search_tool, prompt=prompt, memory=memory)
agent_executor_memory = AgentExecutor(agent=search_agent_memory, tools=search_tool, memory=memory, verbose=False)

In [94]:
agent_executor_memory.run("How many people live in canada as of 2023?")

"Jun 19, 2023 ... As of June 16, 2023, there are now 40 million Canadians! This is a historic milestone for Canada and certainly cause for celebration. It is also\xa0... Jun 28, 2023 ... Canada's population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023. This was the\xa0... The current population of Canada is 38,808,843 as of Tuesday, August 1, 2023, based on Worldometer elaboration of the latest United Nations data 1. Canada 2023\xa0... Mar 22, 2023 ... Record-high population growth in the year 2022. Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population\xa0... Jun 28, 2023 ... Index to the latest information from the Census of Population. ... 2023. Census in Brief: English–French bilingualism in Canada: Recent\xa0... Jul 20, 2023 ... During the 1960's, aerial surveys identified the Ungava Peninsula in northern Québec as the primary nesting area for Atlantic flyway Canada\xa0... The curren

In [96]:
agent_executor_memory.run("What is the population of its largest provence?")

"Its four largest provinces by area (Ontario, Quebec, British Columbia, and Alberta) are also its most populous; together they account for 86.5% of the country's\xa0... Only three of Canada's ten provinces have populations of fewer than 1 million people. Ontario is the largest, boasting a population of more than 15 million\xa0... Ontario and Quebec have always been the two biggest provinces in Canada, with together over 60% of the population at any given time. The population of the West\xa0... Nov 22, 2022 ... Single population in Canada in 2022, by province ; Manitoba, 681,723 ; Saskatchewan, 566,419 ; Nova Scotia, 450,742 ; New Brunswick, 350,480. A population centre, in the context of a Canadian census, is a populated place, or a cluster of interrelated populated places, which meets the demographic\xa0... Feb 8, 2017 ... In 2016, 6.6% of Canadians lived in the Atlantic provinces, with Nova Scotia (2.6%) accounting for the largest share, followed by New\xa0... Jul 16, 2023 ... Quebec

Looking at the [trace](https://smith.langchain.com/public/537b4a4b-35fa-409e-9a5b-d7f40e3b9772/r), we can see that the chat history is persisted.

### Long-term memory 

Vectorstores are great options for long-term memory.

In [24]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
embedding_size = 1536
embeddings_model = OpenAIEmbeddings()
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

### Going deeper 

* Explore projects using long-term memory, such as [autonomous agents](https://python.langchain.com/docs/use_cases/autonomous_agents/).

## Tools 

--- 

As mentioned above, LangChain has [many tools](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/load_tools.py) for Agents that we can load easily.

We can also define [custom tools](https://python.langchain.com/docs/modules/agents/tools/custom_tools). For example, here is a search tool.

The `Tool` dataclass wraps functions that accept a single string input and returns a string output.

`return_direct` determines whether to return the tool's output directly. 

Setting this to `True` means that after the tool is called, the `AgentExecutor` will stop looping.

In [ ]:
from langchain.agents import tool
from langchain.utilities import GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper()
search_tool = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True,
    )
]

To make it easier to define custom tools, a @tool decorator is provided. 

This decorator can be used to quickly create a Tool from a simple function.

In [104]:
# Tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)
word_length_tool = [get_word_length]

### Going deeper

**Toolkits**

Toolkits are groups of tools needed to accomplish specific objectives.

[Here](https://python.langchain.com/docs/integrations/toolkits/) are > 15 different agent toolkits (e.g., Gmail, Pandas, etc).   

## Agents

--- 

There's a number of [agent types](https://python.langchain.com/docs/modules/agents/agent_types/) available in LangChain.

**Action agents** 

Use an LLM to determine which actions to take and in what order:

* [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react.html): This is the most general purpose action agent using the [ReAct framework](https://arxiv.org/pdf/2205.00445.pdf), which can work with [Docstores](https://python.langchain.com/docs/modules/agents/agent_types/react_docstore.html) or [Multi-tool Inputs](https://python.langchain.com/docs/modules/agents/agent_types/structured_chat.html).
* [OpenAI functions](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent.html): Designed to work with OpenAI function-calling models.
* [Conversational](https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent.html): This agent is designed to be used in conversational settings
* [Self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search.html): Designed to lookup factual answers to questions

**Plan-and-execute agents** 

Plan and execute agents accomplish an objective by first planning what to do, then executing the sub tasks. 

* [BabyAGI](https://github.com/yoheinakajima/babyagi), which has a LangChain [user](https://python.langchain.com/docs/use_cases/agents/baby_agi) [guide](https://python.langchain.com/docs/use_cases/agents/baby_agi_with_agent)

### OpenAI Functions agent

As shown in Quickstart, let's continue with [`OpenAI functions` agent](https://python.langchain.com/docs/modules/agents/agent_types/).

This uses OpenAI models, which are fine-tuned to detect when a function should to be called.

They will respond with the inputs that should be passed to the function.

But, we can unpack it, first with a custom prompt:

In [103]:
# Memory
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

# Prompt
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder
system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

Define agent:

In [105]:
# Agent 
from langchain.agents import OpenAIFunctionsAgent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
agent = OpenAIFunctionsAgent(llm=llm, tools=word_length_tool, prompt=prompt)

Run agent:

In [106]:
# Run the executer, including short-term memory we created
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=False)
agent_executor.run("how many letters in the word educa?")

'There are 5 letters in the word "educa".'

### ReAct

If we are working with an LLM that does not support functions, [ReAct](https://arxiv.org/abs/2210.03629) agents are a popular choice.

You can see the charecteristic `Thought`, `Action`, `Observation` [pattern in the output](https://lilianweng.github.io/posts/2023-06-23-agent/).

Also note that we can use `initialize_agent` to easily create the agent.

Browse all the agent types [here](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/types.py).

In [111]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [116]:
MEMORY_KEY = "chat_history"
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)
react_agent = initialize_agent(search_tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False, memory=memory)

In [ ]:
react_agent("How many people live in canada as of 2023?")

In [ ]:
react_agent("What is its largest province?")

The LangSmith [trace](https://smith.langchain.com/public/d3bbfa9a-e518-473f-8167-1d8690ecc38a/r) gives us insight into the ReAct prompt.

![Image description](/img/agents_use_case_trace_2.png)

### Custom

Let's peel it back even further to define our own action agent.

We can [create a custom agent](https://python.langchain.com/docs/modules/agents/how_to/custom_agent.html) to unpack the central pieces:

* `Tools`: The tools the agent has available to use
* `Agent`: decides which action to take

In [34]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent

class FakeAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")
    
fake_agent = FakeAgent()
fake_agent_executor = AgentExecutor.from_agent_and_tools(agent=fake_agent, 
                                                         tools=search_tool, 
                                                         verbose=False)

fake_agent_executor.run("How many people live in canada as of 2023?")

"The current population of Canada is 38,808,843 as of Tuesday, August 1, 2023, based on Worldometer elaboration of the latest United Nations data 1. Canada 2023\xa0... Mar 22, 2023 ... Record-high population growth in the year 2022. Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population\xa0... Jun 19, 2023 ... As of June 16, 2023, there are now 40 million Canadians! This is a historic milestone for Canada and certainly cause for celebration. It is also\xa0... Jun 28, 2023 ... Canada's population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023. The main driver of population growth is immigration, and to a lesser extent, natural growth. Demographics of Canada · Population pyramid of Canada in 2023. May 2, 2023 ... On January 1, 2023, Canada's population was estimated to be 39,566,248, following an unprecedented increase of 1,050,110 people between January\xa0... Canada ranks 37th by population

## Going deeper

* Explore the other agent types [here](https://python.langchain.com/docs/modules/agents/agent_types/)